In [1]:
import numpy as np
import pandas as pd

## Kumpulan Matrix

### Ruas kiri dan kanan jadi 1 matrix

In [ ]:
# 3x + y - z = 5
# 4x + 7z - 3z = 20
# 2x -2y + 5z = 10

matrix = np.array([[3.,  1., -1.,  5.],
                   [4.,  7., -3., 20.],
                   [2., -2.,  5., 10.]])

# hasil = [1.5060241  3.13253012 2.65060241]

In [ ]:
matrix = np.array([[ 1., 1.,  1.,  1., 13.],
                   [ 2., 3.,  0., -1.,  1.],
                   [-3., 4.,  1.,  2., 10.],
                   [ 1., 2., -1.,  1.,  1.]])

# hasil = [2.11111111 0.37037037 6.18518519 4.33333333]

### Ruas kiri matrix A, ruas kanan matrix B

In [ ]:
A = np.array([[3., 1., -1.],
              [4., 7., -3.],
              [2., -2., 5.]])

B = np.array([[ 5.], 
              [20.], 
              [10.]])

# hasil = [1.5060241  3.13253012 2.65060241]

In [ ]:
# |   1   1   1   1 | | x |   | 13 |
# |   2   3   0  -1 | | y | = |  1 |
# |  -3   4   1   2 | | z |   | 10 |
# |   1   2  -1   1 | | w |   |  1 |

A = np.array([[ 1.,  1.,  1.,  1.],
              [ 2.,  3.,  0., -1.],
              [-3., 4.,  1.,  2.],
              [ 1.,  2., -1.,  1.]])

B = np.array([[13.],
              [ 1.],
              [10.],
              [ 1.]])

# hasil = [2.11111111 0.37037037 6.18518519 4.33333333]

## Metode Eliminasi Gauss

### Manual

In [101]:
matrix = np.array([[3., 1., -1., 5.],
                   [4., 7., -3., 20.],
                   [2., -2., 5., 10.]])

size = matrix[:, 0].size

for i in range(0, size-1):
    matrix[i] = matrix[i]/matrix[i][i]
    for j in range(i, size-1):
        matrix[j+1] = matrix[j+1] - matrix[j+1][i]*matrix[i]

matrix[size-1] = matrix[size-1]/matrix[size-1][size-1]

# sampai sini sudah membentuk segitiga atas

for i in range(0, size-1):
    ii = size-i-2  # ... 2 1 0
    for j in range(i, size-1):
        jj = size-j-2 # ... 2 1 0
        matrix[jj][size] -= matrix[jj][ii+1] * matrix[ii+1][size]
        matrix[jj][ii+1] = 0
        
# malah jadi gauss-jordan :)

print("matrix", matrix, sep="\n")

hasil = matrix[:, size]
print("\nhasil", hasil, sep="\n")

matrix
[[1.         0.         0.         1.5060241 ]
 [0.         1.         0.         3.13253012]
 [0.         0.         1.         2.65060241]]

hasil
[1.5060241  3.13253012 2.65060241]


### Numpy `np.linalg.solve(A, B)`

In [70]:
# cara magic
# https://radiokuantum.wordpress.com/2018/03/22/menyelesaikan-persamaan-linear-multi-variabel-dengan-menggunakan-python-dan-numpy/

# |   1   1   1   1 | | x |   | 13 |
# |   2   3   0  -1 | | y | = |  1 |
# |  -3   4   1   2 | | z |   | 10 |
# |   1   2  -1   1 | | w |   |  1 |

A = np.array([[1, 1, 1, 1],
              [2, 3, 0, -1],
              [-3, 4, 1, 2],
              [1, 2, -1, 1]])

B = np.array([[13],
              [1],
              [10],
              [1]])

x = np.linalg.solve(A, B)
print(x)

[[2.11111111]
 [0.37037037]
 [6.18518519]
 [4.33333333]]
